In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
X = application_df.copy().drop(columns=["EIN", "NAME"])
X.columns

In [ ]:
# Determine the number of unique values in each column.
X.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = X['APPLICATION_TYPE'].value_counts()
application_type_counts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type_counts[application_type_counts < 500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = X['CLASSIFICATION'].value_counts()
classification_counts

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts[classification_counts < 500].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
application_df.head()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
X2_df = pd.get_dummies(X)
X2_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
X = X2_df.drop(columns=["IS_SUCCESSFUL"]).values
y = X2_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Compile, Train and Evaluate the Model

In [ ]:
# Determine the number of input features
input_features = X_train.shape[1]  # Number of columns in X_train

print(f"Number of input features: {input_features}")

Model B

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def create_model_b():
    nn_b = tf.keras.models.Sequential()
    nn_b.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=116))
    nn_b.add(tf.keras.layers.Dense(units=50, activation='relu'))
    nn_b.add(tf.keras.layers.Dense(units=25, activation='relu'))
    nn_b.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    return nn_b

# Create the model
model_b = create_model_b()

# Check the structure of the model
model_b.summary()

In [ ]:
# Compile model B
model_b.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train model B
history_b = model_b.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test))

In [ ]:
# Evaluate model B using the test data
model_loss_b, model_accuracy_b = model_b.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export model B to HDF5 file
model_b.save("AlphabetSoupCharity_OptimizationB.h5")

Model C

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def create_model_c():
    nn_c = tf.keras.models.Sequential()
    nn_c.add(tf.keras.layers.Dense(units=64, activation='relu', input_dim=116, kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    nn_c.add(tf.keras.layers.Dropout(0.2))
    nn_c.add(tf.keras.layers.Dense(units=32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    nn_c.add(tf.keras.layers.Dropout(0.2))
    nn_c.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    return nn_c

# Create the model
model_c = create_model_c()

# Check the structure of the model
model_c.summary()

In [ ]:
# Compile model C
model_c.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train model C
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history_c = model_c.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

In [ ]:
# Evaluate model C using the test data
model_loss_c, model_accuracy_c = model_c.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export model C to HDF5 file
model_c.save("AlphabetSoupCharity_OptimizationC.h5")